In [13]:
from pathlib import Path
import pandas as pd
import plotly.express as px



In [21]:
results = Path("results/att_sens").glob("**/*shares_df.csv")

dfs = []

for r_path in results:
    parent_dir = r_path.parent.stem
    parent_dir = parent_dir.replace("CER_plus","CER+").replace("Rapid_plus","Rapid+")
    scenario,*_, att_case = parent_dir.split("_")
    df = pd.read_csv(r_path)
    df["scenario"] = scenario
    df["attCase"] = att_case
    dfs.append(df.drop("province", axis=1))

adoption_df = pd.concat(dfs)

In [24]:
agg_adoption_df = adoption_df.groupby(["year","scenario","attCase"]).mean().drop("RunId", axis=1).melt(ignore_index=False)
agg_adoption_df.head()


variable  value
year   scenario attCase                     
2020.0 BAU      fast      Gas furnace  0.714
                moderate  Gas furnace  0.714
                no        Gas furnace  0.714
                slow      Gas furnace  0.714
       CER      fast      Gas furnace  0.714

In [27]:
from config import TECHNOLOGY_COLORS
fig = px.line(
    agg_adoption_df.reset_index(),
    x="year",
    y="value",
    color="variable",
    facet_col="scenario",
    facet_row="attCase",
    category_orders={"attCase": ["no", "slow", "moderate", "fast"]},
    color_discrete_map=TECHNOLOGY_COLORS
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))